In [ ]:
import os
import sys
import re
import time
from github import Github
from random import randint
from pprint import pprint

# Credentials

In [ ]:
from pymongo import MongoClient
pw_file = 'credentials/pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        email, indeed_pw = f.readline().strip().split(', ')
        username, pia_pw = f.readline().strip().split(', ')
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

# Connect to MongoDB

In [ ]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

# Create a MongoDB collection

In [ ]:
for name in db.collection_names():
    #db.drop_collection(name)
    print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [ ]:
# create a collection in the 'github_db' database
lst_collections = ['git_users_meta', 'git_users_following', 'git_users_followers', 'git_users_starred', 
                   'git_repos_meta', 'git_repos_docs', 'git_repos_subscribers', 'git_repos_stargazers', 
                  'git_repos_contributors']

for collection in lst_collections:
    if collection not in db.collection_names():
        db.create_collection(collection)

In [ ]:
for name in db.collection_names():
    print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

# Get authenticated access to Github for higher requests limit

In [ ]:
# import github private token
with open('credentials/token.txt', 'r') as infile:
    token = infile.readline().strip()
    user = infile.readline().strip()

git_client = Github(token)

# Rate limits

In [ ]:
def get_remaining(_type ='core'):
    '''
    _type = 'core' or 'search'
    Return: tuple of remaining rate limit quantity, and time till reset
    '''
    
    rate_limit = git_client.get_rate_limit()
    raw = dict(rate_limit.raw_data)
    
    remaining = int(raw['resources'][_type]['remaining'])
    reset = int(raw['resources'][_type]['reset'])
    
    return (remaining, reset)

In [ ]:
print('Resource rate limit remaining: {0}'.format(get_remaining('core')))
print('Search rate limit remaining: {0}'.format(get_remaining('search')))

# Wait function

In [ ]:
def limit_wait(num_files=10):
    # wait till rate limit is refreshed
    rate, _ = get_remaining('core')
    
    ct = 0
    while (rate < num_files):
        print('Waiting.....{0}'.format(ct))
        time.sleep(300)
        rate, _ = get_remaining('core')
        ct += 5

# X X X X X X X X X X X X X X X X X X X X X X X X X X 

# Repo Metadata

In [ ]:
def repo_metadata(repo):
    repo_keys = ['owner', 'name', 'full_name', 'description', 'fork', 'html_url', 'homepage', 
             'language', 'forks_count', 'size', 'open_issues_count', 'has_issues', 'has_wiki', 'has_downloads', 
             'pushed_at', 'created_at', 'subscribers_count', 'stargazers_count']
    
    owner_keys = ['type', 'login', 'id', 'site_admin']
    
    # check for rate limit
    limit_wait(num_files=10)
    
    # get returned payload raw data into dictionary
    raw_dict = dict(repo.raw_data)
    
    # trim repo by keys listed above
    repo_dict = {k: raw_dict[k] for k in repo_keys}
    
    # trim owner by keys listed above
    owner_dict = {k: raw_dict['owner'][k] for k in owner_keys}
    repo_dict['owner'] = owner_dict
    repo_dict['_id'] = repo.id
    
    return repo_dict

In [ ]:
def repo_metadata_upsert(repo):
    col = db['git_repos_meta']
    
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set':{col.insert_one(repo_metadata(repo))}}, 
            upsert=True)
    except:
        pass
    return None

# Repo Subscribers

In [ ]:
def repo_subscribers(repo):
    '''
    IN: github repo object
    RETURN: dictionary containing repo subscribers
    '''
    repo_dict = {}
    
    # check for rate limit
    limit_wait(num_files=10)
    
    repo_dict['_id'] = repo.id
    repo_dict['full_name'] = repo.full_name
    
    try:
        repo_dict['repo_subscribers'] = [f.login for f in repo.get_subscribers()]
    except:
        repo_dict['repo_subscribers'] = []
        
    return repo_dict

In [ ]:
def repo_subscribers_upsert(repo):
    col = db['git_repos_subscribers']
    
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set': {col.insert_one(repo_subscribers(repo))}
            }, upsert=True)
    except:
        pass
    return None

# Repo Stargazers

In [ ]:
def repo_stargazers(repo):
    '''
    IN: github repo object
    RETURN: dictionary containing list of stargazers
    '''
    repo_dict = {}
    
    # check for rate limit
    limit_wait(num_files=10)

    repo_dict['_id'] = repo.id
    repo_dict['full_name'] = repo.full_name
    
    try:
        repo_dict['repo_stargazers'] = [f.login for f in repo.get_stargazers()]
    except:
        repo_dict['repo_stargazers'] = []

    return repo_dict

In [ ]:
def repo_stargazers_upsert(repo):
    col = db['git_repos_stargazers']
    
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set': {col.insert_one(repo_stargazers(repo))}
            }, upsert=True)
    except:
        pass
    return None

# Repo Contributors

In [ ]:
def repo_contributors(repo):
    '''
    IN: github repo object
    RETURN: dictionary of contributors keyed by repo full_name
    '''
    repo_dict = {}
    
    # check for rate limit
    limit_wait(num_files=10)
    
    repo_dict['_id'] = repo.id
    repo_dict['full_name'] = repo.full_name
    
    try:        
        repo_dict['repo_contributors'] = [f.login for f in repo.get_contributors()]
    except:
        repo_dict['repo_contributors'] = ''
        
    return repo_dict

In [ ]:
def repo_contributors_upsert(repo):
    col = db['git_repos_contributors']
    
    try:
        col.update_one(
            {'_id': repo.id}, 
            {'$set': {col.insert_one(repo_contributors(repo))}
            }, upsert=True)
    except:
        pass
    return None

# User Metadata

In [ ]:
def user_metadata(user):
    user_keys = ['email', 'followers', 'hireable', 'login', 'bio', 'avatar_url', 'company', 
                 'updated_at', 'type', 'created_at', 'name', 'location', 'html_url', 'public_repos', 
                 'blog', 'public_gists', 'following']
    # check for rate limit
    limit_wait(num_files=10)
        
    # get returned payload raw data into dictionary
    raw_dict = dict(user.raw_data)
    
    # trim user by keys listed above
    user_dict = {k: raw_dict[k] for k in user_keys}
    user_dict['_id'] = user.id
    
    return user_dict

In [ ]:
def user_metadata_upsert(user):
    col = db['git_users_meta']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set':{col.insert_one(user_metadata(user))}}, 
            upsert=True)
    except:
        pass
    return None

# User Following

In [ ]:
def user_following(user):
    '''
    IN: github user object
    RETURN: dictionary of users (login name) followed by user (passed) 
            keyed by passed user's login
    '''
    user_dict = {}
    
    # check for rate limit
    limit_wait(num_files=10)

    user_dict['_id'] = user.id
    user_dict['login'] = user.login
        
    try:
        user_dict['user_following'] = [f.name for f in user.get_following()]
    except:
        user_dict['user_following'] = []
        
    return user_dict

In [ ]:
def user_following_upsert(user):
    col = db['git_users_following']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_following(user))}
            }, upsert=True)
    except:
        pass
    return None

# User Followers

In [ ]:
def user_followers(user):
    '''
    IN: github user object
    RETURN: dictionary of users (login name) who follow user (passed)
            keyed by passed user's login
    '''
    user_dict = {}
    
    # check for rate limit
    limit_wait(num_files=10)
    
    user_dict['_id'] = user.id
    user_dict['login'] = user.login
        
    try:
        user_dict['user_followers'] = [f.login for f in user.get_followers()]
    except:
        user_dict['user_followers'] = []
    
    return user_dict

In [ ]:
def user_followers_upsert(user):
    col = db['git_users_followers']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_followers(user))}
            }, upsert=True)
    except:
        pass
    return None

# Users Starred

In [ ]:
def user_starred(user):
    '''
    IN: github user object
    RETURN: dictionary of repos (full_name) starred by user (passed)
            keyed by passed user's login
    '''
    user_dict = {}
    
    # check for rate limit
    limit_wait(num_files=10)
    
    user_dict['_id'] = user.id
    user_dict['login'] = user.login
    
    try:
        user_dict['starred_repos'] = [f.full_name for f in user.get_starred()]
    except:
        user_dict['starred_repos'] = []
        
    return user_dict

In [ ]:
def user_starred_upsert(user):
    col = db['git_users_starred']
    
    try:
        col.update_one(
            {'_id': user.id}, 
            {'$set': {col.insert_one(user_starred(user))}
            }, upsert=True)
    except:
        pass
    return None

# Repo Files

In [ ]:
def repo_scripts_upsert(repo, file_name, raw_file_cont, file_ext, doc_name):
    '''
    '''
    # strip newline and tab chars from raw file content
    col = db['git_repos_docs']
    doc_name = '{0}_{1}'.format(repo.id, doc_name)
    strip_file_cont = raw_file_cont.replace('\n', ' ')
    strip_file_cont = strip_file_cont.replace('\t', ' ')

    try:
        col.update_one(
            {'document' : doc_name}, 
            {'$set':{
                    'file_name': file_name,
                    'raw_file': raw_file_cont,
                    'strip_file': strip_file_cont,
                    'extension': file_ext,
                    'full_name' : repo.full_name,
                    'document' : doc_name}
                }, 
            upsert=True)
    except:
        pass
    return None

In [ ]:
def repo_scripts(repo, doc_ct=0, _path='.'):
    '''
    IN: github repo object and path to folder
    RETRUN: all files from repo except those in the exclusion list
    '''
    # only files with these extensions
    extensions = ['py', 'md']
    
    # grab all contents in the main directory
    dir_contents = repo.get_dir_contents(_path)
    num_files = len(dir_contents)
    repo_fullname = repo.full_name

    # check for rate limit
    limit_wait(num_files=num_files)
    
    for content in dir_contents:
        # if item is a directory then recursively navigate lower to get files inside
        if content.type == 'dir':
            repo_scripts(repo, doc_ct, _path=content.path)

        else:
            doc_ct += 1
            # get file extension
            file_ext = content.name.split('.')[-1]

            if file_ext in extensions:
                # try to decode, but return blank if fail
                try:
                    raw_file_content = content.decoded_content.decode(errors='replace')
                except:
                    raw_file_content = ''
                    
                # actual file name, mongodb doc stored name
                file_name = content.name
                doc_name = 'doc_{0}'.format(doc_ct)
                
                # add to collection if under limit (>16mb)
                file_size = sys.getsizeof(raw_file_content)
                if file_size < 16000000:
                    repo_scripts_upsert(repo, file_name, raw_file_content, file_ext, doc_name)
                else:
                    print('FILE SIZE LIMIT: {0} -- {1}'.format(file_name, file_size))
    return None

# Search Github for 'Python' repositories

In [ ]:
ct = 0
for repo in git_client.search_repositories('python', sort='stars', order='desc'):
    ct+=1

    try:
        # ensure its a python tagged dirctory
        if len(repo.language.lower()) >1 and repo.language.lower() == 'python':

            #############################
            # INSERT FURTHER CALLS HERE #

            if db['git_repos_meta'].find({'_id': repo.id}).count() == 0:
                print('Starting.....#{0} - {1}'.format(ct, repo.full_name))
                repo_scripts(repo)

                # insert metadata
                repo_metadata_upsert(repo)
                repo_stargazers_upsert(repo)
                repo_subscribers_upsert(repo)
                repo_contributors_upsert(repo)

            user = repo.owner
            if db['git_users_meta'].find({'_id': user.id}).count() == 0:
                user_metadata_upsert(user)
                user_following_upsert(user)
                user_followers_upsert(user)
                user_starred_upsert(user)
            #############################

        # don't get cut off
        time.sleep(randint(2,9))
    
    except:
        pass

    
print('DONE')

In [ ]:
print('Resource rate limit remaining: {0}'.format(get_remaining('core')))
print('Search rate limit remaining: {0}'.format(get_remaining('search')))
print()
for name in sorted(db.collection_names()):
    print('Number of documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [ ]:
for cursor in db['git_users_meta'].find():
    pprint(cursor)